# DeepLoc

### Get train.fasta and test.fasta

In [1]:
%%capture
!pip install Bio

In [10]:
import numpy as np
import pandas as pd
import itertools
from itertools import chain
import json
import argparse
import tqdm
from Bio import SeqIO
import re
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [4]:
file1 = open('/content/deeploc_data.fasta', 'r')
Lines = file1.readlines()
for i in range(len(Lines)):
  if('test' in Lines[i] and '>' in Lines[i]):
    f2 = open("deeploc_test.fasta", "a")
    f2.write(Lines[i])
    f2.write(Lines[i+1])
    f2.close()
  if('test' not in Lines[i] and '>' in Lines[i]):
    f1 = open("deeploc_train.fasta", "a")
    f1.write(Lines[i])
    f1.write(Lines[i+1])
    f1.close()

### Homology Partitioning

In [5]:
train_sequences = SeqIO.parse(open('/content/deeploc_train.fasta'),'fasta')
test_sequences = SeqIO.parse(open('/content/deeploc_test.fasta'),'fasta')

id_train = []
s_train = []
d_train = []

for fasta in train_sequences:
  name, desc, sequence = fasta.id,fasta.description, str(fasta.seq)
  if(re.search(r'[^ACDEFGHIKLMNPQRSTVWY]+', sequence, flags=0)!=None):
    sequence = sequence.replace('B','').replace('X','').replace('Z','').replace('U','')
  s_train.append(sequence)
  d_train.append(desc)
  id_train.append(name)

id_test = []
s_test = []
d_test = []

for fasta in test_sequences:
  name, desc, sequence = fasta.id,fasta.description, str(fasta.seq)
  if(re.search(r'[^ACDEFGHIKLMNPQRSTVWY]+', sequence, flags=0)!=None):
    sequence = sequence.replace('B','').replace('X','').replace('Z','').replace('U','')
  s_test.append(sequence)
  d_test.append(desc)
  id_test.append(name)

print(len(id_train))
print(len(id_test))

11231
2773


In [6]:
file1 = open('/content/output.clstr', 'r')
Lines = file1.readlines()
print(len(Lines))
all_clstr = []

n = 0
for l in Lines:
  if('Cluster 0' in l): #clstr 0
    tmp = []
  if(('Cluster' in l and 'Cluster 0' not in l)):
    all_clstr.append(tmp)
    tmp = []
  if('Cluster' not in l):
    id_clstr = l.split()[2].replace('>','').replace('...','')
    tmp.append(id_clstr)
    if(id_clstr in Lines[-1]):
      all_clstr.append(tmp)
      break

18758


In [7]:
d1 = {'Nucleus':0,'Cytoplasm':1,'Extracellular':2,'Mitochondrion':3,'Cell.membrane':4,
     'Endoplasmic.reticulum':5,'Plastid':6,'Golgi.apparatus':7,'Lysosome/Vacuole':8,'Peroxisome':9}
y_all = []
X_all = []
for cluster in all_clstr:
  y_tmp = []
  x_tmp = []
  for pro_id in cluster:
    y = d_train[id_train.index(pro_id)]
    y = re.split(r'\s|-', y)[-2]
    y = d1[y]
    X = s_train[id_train.index(pro_id)]
    x_tmp.append(X)
    y_tmp.append(y)
  y_all.append(y_tmp)
  X_all.append(x_tmp)

Partition train set into 4 group with label ratio approximately same as test set

In [ ]:
y_test = [re.split(r'\s|-', i)[-3] for i in d_test]
y_test = [d1[i] for i in y_test]
print( pd.Series(y_test).value_counts())
ratio = np.array([i/30 for i in pd.Series(y_test).value_counts().values])

Find best randomstate to split

In [ ]:
# import pandas as pd
# from sklearn.model_selection import StratifiedKFold, train_test_split
# from itertools import chain

# best_dis = 1000

# for k in range(10000):
#   X_train, X_f1, y_train, y_f1 = train_test_split(X_all, y_all, test_size=0.25, random_state=8847)
#   X_train, X_f2, y_train, y_f2 = train_test_split(X_train, y_train, test_size=0.3333, random_state=91)
#   X_f4, X_f3, y_f4, y_f3 = train_test_split(X_train, y_train, test_size=0.5, random_state=k)
#   stat1 = pd.Series(list(chain.from_iterable(y_f1))).value_counts().sort_index().values
#   stat2 = pd.Series(list(chain.from_iterable(y_f2))).value_counts().sort_index().values
#   stat3 = pd.Series(list(chain.from_iterable(y_f3))).value_counts().sort_index().values
#   stat4 = pd.Series(list(chain.from_iterable(y_f4))).value_counts().sort_index().values

#   r1 = np.array([i/min(stat1) for i in stat1])
#   r2 = np.array([i/min(stat2) for i in stat2])
#   r3 = np.array([i/min(stat3) for i in stat3])
#   r4 = np.array([i/min(stat4) for i in stat4])

#   dis1 = np.abs(ratio - r1)
#   dis2 = np.abs(ratio - r2)
#   dis3 = np.abs(ratio - r3)
#   dis4 = np.abs(ratio - r4)
#   dis_all = (dis1 + dis2 + dis3 + dis4)/4
#   dis_all = np.average(dis_all)
#   if(dis_all<best_dis):
#     print(8847,91,k,'-',dis_all)
#     best_dis=dis_all
#     best_randomstate = [8847,91,k]
#   print(len(list(chain.from_iterable(y_f1))))
#   print(len(list(chain.from_iterable(y_f2))))
#   print(len(list(chain.from_iterable(y_f3))))
#   print(len(list(chain.from_iterable(y_f4))))

8847 91 0 - 0.4878640039859102
8847 91 3 - 0.4496051747311827
8847 91 23 - 0.41715893817204297
8847 91 34 - 0.4000890456989247
8847 91 57 - 0.38678259408602156
8847 91 116 - 0.3686374327956989
8847 91 123 - 0.3521858198924731
8847 91 141 - 0.3380729166666666


In [16]:
X_train, X_f1, y_train, y_f1 = train_test_split(X_all, y_all, test_size=0.25, random_state=8847)
X_train, X_f2, y_train, y_f2 = train_test_split(X_train, y_train, test_size=0.3333, random_state=91)
X_f4, X_f3, y_f4, y_f3 = train_test_split(X_train, y_train, test_size=0.5, random_state=141)

stat1 = pd.Series(list(chain.from_iterable(y_f1))).value_counts().sort_index().values
stat2 = pd.Series(list(chain.from_iterable(y_f2))).value_counts().sort_index().values
stat3 = pd.Series(list(chain.from_iterable(y_f3))).value_counts().sort_index().values
stat4 = pd.Series(list(chain.from_iterable(y_f4))).value_counts().sort_index().values

print(stat1)
print(stat2)
print(stat3)
print(stat4)

print(len(list(chain.from_iterable(y_f1))))
print(len(list(chain.from_iterable(y_f2))))
print(len(list(chain.from_iterable(y_f3))))
print(len(list(chain.from_iterable(y_f4))))

[861 497 416 338 245 194 154  69  61  32]
[839 527 389 308 269 168 159  59  63  31]
[834 503 386 306 274 153 143  84  65  30]
[847 507 389 256 279 174 149  74  68  31]
2867
2812
2778
2774


In [29]:
Xf1 = list(chain.from_iterable(X_f1))
Xf2 = list(chain.from_iterable(X_f2))
Xf3 = list(chain.from_iterable(X_f3))
Xf4 = list(chain.from_iterable(X_f4))

yf1 = list(chain.from_iterable(y_f1))
yf2 = list(chain.from_iterable(y_f2))
yf3 = list(chain.from_iterable(y_f3))
yf4 = list(chain.from_iterable(y_f4))

X_train = Xf1+Xf2+Xf3+Xf4
y_train = yf1+yf2+yf3+yf4
p_train = [1]*len(Xf1)+[2]*len(Xf2)+[3]*len(Xf3)+[4]*len(Xf4)

X_test = s_test
y_test = y_test

### Protein Sequence Preprocessing

In [30]:
blosum62_rownorm = pd.read_csv('/content/blosum62_freqrow.csv',header=0,delimiter='^')
blosum62_rownorm = blosum62_rownorm.set_axis(blosum62_rownorm.columns)
blosum62_rownorm 

,A,R,N,D,C,Q,E,G,H,I,L,K,M,F,P,S,T,W,Y,V
A,0.2901,0.0310,0.0256,0.0297,0.0216,0.0256,0.0405,0.0783,0.0148,0.0432,0.0594,0.0445,0.0175,0.0216,0.0297,0.0850,0.0499,0.0054,0.0175,0.0688
R,0.0446,0.3450,0.0388,0.0310,0.0078,0.0484,0.0523,0.0329,0.0233,0.0233,0.0465,0.1202,0.0155,0.0174,0.0194,0.0446,0.0349,0.0058,0.0174,0.0310
N,0.0427,0.0449,0.3169,0.0831,0.0090,0.0337,0.0494,0.0652,0.0315,0.0225,0.0315,0.0539,0.0112,0.0180,0.0202,0.0697,0.0494,0.0045,0.0157,0.0270
D,0.0410,0.0299,0.0690,0.3974,0.0075,0.0299,0.0914,0.0466,0.0187,0.0224,0.0280,0.0448,0.0093,0.0149,0.0224,0.0522,0.0354,0.0037,0.0112,0.0243
C,0.0650,0.0163,0.0163,0.0163,0.4837,0.0122,0.0163,0.0325,0.0081,0.0447,0.0650,0.0203,0.0163,0.0203,0.0163,0.0407,0.0366,0.0041,0.0122,0.0569
Q,0.0559,0.0735,0.0441,0.0471,0.0088,0.2147,0.1029,0.0412,0.0294,0.0265,0.0471,0.0912,0.0206,0.0147,0.0235,0.0559,0.0412,0.0059,0.0206,0.0353
E,0.0552,0.0497,0.0405,0.0902,0.0074,0.0645,0.2965,0.0350,0.0258,0.0221,0.0368,0.0755,0.0129,0.0166,0.0258,0.0552,0.0368,0.0055,0.0166,0.0313
G,0.0783,0.0229,0.0391,0.0337,0.0108,0.0189,0.0256,0.5101,0.0135,0.0189,0.0283,0.0337,0.0094,0.0162,0.0189,0.0513,0.0297,0.0054,0.0108,0.0243
H,0.0420,0.0458,0.0534,0.0382,0.0076,0.0382,0.0534,0.0382,0.3550,0.0229,0.0382,0.0458,0.0153,0.0305,0.0191,0.0420,0.0267,0.0076,0.0573,0.0229
I,0.0471,0.0177,0.0147,0.0177,0.0162,0.0133,0.0177,0.0206,0.0088,0.2710,0.1679,0.0236,0.0368,0.0442,0.0147,0.0250,0.0398,0.0059,0.0206,0.1767


In [32]:
def encode1(protein):
  p = []
  for acid in protein:
    p.append(blosum62_rownorm.loc[acid].values.tolist())
  return p

def remove_middle(s,max_len):
  a = s
  while(len(a)>max_len):
    tmp = list(a)
    tmp[int(len(tmp)/2)]=''
    a = "".join(tmp)
  return a

def padding(arr):
  shape = np.shape(arr)
  padded_array = np.zeros((1000, 20))
  padded_array[:shape[0],:shape[1]] = arr
  return padded_array

def masking(len_seq):
  a = np.zeros(1000)
  if(len_seq<1000):
    a[:len_seq]=np.ones(len_seq)
  return a

def preprocess(list_of_seq):
  seq_result = []
  mask_result = []

  for seq in tqdm.tqdm_notebook(list_of_seq):
    if(len(seq)>1000):
      seq1 = remove_middle(seq,1000)#string
      seq1 = encode1(seq1) #np.array 1000x20
    else:
      seq1 = encode1(seq) ##np.array nx20
      seq1 = padding(seq1) #padding with 0 #np.array 1000x20

    mask_seq = masking(len(seq))
    seq_result.append(seq1)
    mask_result.append(mask_seq)
    
  return np.array(seq_result,dtype=np.float32),np.array(mask_result,dtype=np.float32)

In [34]:
X_train_enc, mask_train = preprocess(X_train)
X_test_enc, mask_test = preprocess(X_test)

y_train_n = np.array(y_train,dtype=np.int32)
y_test_n = np.array(y_test,dtype=np.int32)

### Export

In [35]:
np.savez_compressed('deeploc_train_normr', X_train=X_train_enc,
                                y_train = y_train_n,
                                mask_train = mask_train,
                                partition = p_train)

np.savez_compressed('deeploc_test_normr', X_test=X_test_enc,
                                y_test = y_test_n,
                                mask_test = mask_test)